<a href="https://colab.research.google.com/github/mahuthu/langchain-SQL-chatbot/blob/main/Langchain_sql_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Uncomment the below to use LangSmith. Not required.
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
# os.environ["LANGCHAIN_TRACING_V2"] = "true"

··········


In [11]:
from google.colab import files
uploaded = files.upload()


Saving Chinook.db to Chinook (1).db


In [13]:
from langchain_community.utilities import SQLDatabase

# Path to the uploaded Chinook.db file in Colab
db_file_path = '/content/Chinook (1).db'

# Create a SQLDatabase object using the URI for the SQLite database
db = SQLDatabase.from_uri(f"sqlite:///{db_file_path}")

# Print the database dialect and table names
print(db.dialect)
print(db.get_usable_table_names())

# Execute a query using db.run
result = db.run("SELECT * FROM Artist LIMIT 10;")

# Print the results
for row in result:
    print(row)


sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
[
(
1
,
 
'
A
C
/
D
C
'
)
,
 
(
2
,
 
'
A
c
c
e
p
t
'
)
,
 
(
3
,
 
'
A
e
r
o
s
m
i
t
h
'
)
,
 
(
4
,
 
'
A
l
a
n
i
s
 
M
o
r
i
s
s
e
t
t
e
'
)
,
 
(
5
,
 
'
A
l
i
c
e
 
I
n
 
C
h
a
i
n
s
'
)
,
 
(
6
,
 
'
A
n
t
ô
n
i
o
 
C
a
r
l
o
s
 
J
o
b
i
m
'
)
,
 
(
7
,
 
'
A
p
o
c
a
l
y
p
t
i
c
a
'
)
,
 
(
8
,
 
'
A
u
d
i
o
s
l
a
v
e
'
)
,
 
(
9
,
 
'
B
a
c
k
B
e
a
t
'
)
,
 
(
1
0
,
 
'
B
i
l
l
y
 
C
o
b
h
a
m
'
)
]


In [14]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
db.run(response)

# We can also inspect the chain directly for its prompts. Looking at the prompt

chain.get_prompts()[0].pretty_print()



In [ ]:
# Execute SQL query
## Now that we’ve generated a SQL query, we’ll want to execute it. This is the most dangerous part of creating a SQL chain
## Consider adding a human approval step to you chains before query execution (see below).


In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})